<img src="../../img/change_point_detection.png" alt="change point detection" style="width: 100%; border-radius: 20px;"/>

## Objective
subsequent notebooks: divide into anomaly and drift

## Scope

In [2]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

import pandas as pd
import numpy as np
from skmultiflow.drift_detection import DDM
from skmultiflow.drift_detection.adwin import ADWIN
from river import drift
import plotly.express as px
import plotly.graph_objects as go

from utils.plotting import *
from utils.drift import *

import warnings
warnings.filterwarnings('ignore')

# 
<p style="background-color:#4A3228;color:white;font-size:240%;border-radius:10px 10px 10px 10px;"> &nbsp; 0️⃣ Specify your paths </p>

In order to run the notebook, the following file is required:
- Time series of bird sightings: *[time_series_27_species.csv](https://drive.google.com/drive/folders/18XoTsDtWnN4QdIBNGGbq-jaa6U3nVb2e)*

In [3]:
path_sightings = '../../../01_Data/datasets/time_series_27_species.csv'

# 
<p style="background-color:#4A3228;color:white;font-size:240%;border-radius:10px 10px 10px 10px;"> &nbsp; 1️⃣ Load data </p>

Below are two exemplary time series depicting the sighting frequencies for the pygmy owl and the icterine warbler over the entire observation period from 2018 to 2022 in a 50x50km grid around Basel (CH) and Hannover (DE), respectively. These specific bird species and grids were chosen for the change point analysis due to the pygmy owl exhibiting a habitat drift in the selected grid, while the icterine warbler shows a date drift. 

> Further details about the drifts can be found in the notebooks `02_habitat_drift.ipynb` and `03_date_drift.ipynb`.

In [4]:
df = pd.read_csv(path_sightings)

In [56]:
pygmy_owl = df[(df.name_species == 'Sperlingskauz') & (df.eea_grid_id == '50kmE4100N2700')].reset_index(drop=True)
plot_time_series(data=pygmy_owl, title='Pygmy owl (Sperlingskauz) sightings in grid 50kmE4100N2700')

In [6]:
warbler = df[(df.name_species == 'Gelbspötter') & (df.eea_grid_id == '50kmE4350N3250')].reset_index(drop=True)
plot_time_series(data=warbler, title='Icterine warbler (Gelbspötter) sightings in grid 50kmE4350N3250')

# 
<p style="background-color:#4A3228;color:white;font-size:240%;border-radius:10px 10px 10px 10px;"> &nbsp; 2️⃣ Preprocessing </p>
As can be seen above, the time series are highly noisy and exhibit rapid fluctuations. Therefore, as a preprocessing step, the curves are smoothed to make them more amendable for change point analysis.

Smoothing a volatile, unsmoothed time series as a preprocessing step for change point analysis offers several advantages. Firstly, it mitigates the noise and variability inherent in the raw data, thereby enhancing the identification of meaningful patterns and trends. Consequently, this contributes to a more precise detection of change points, as the smoothed data offers a clearer depiction of the underlying sighting patterns.

To achieve this, we apply a 30-day rolling window to calculate the mean of the time series data, effectively smoothing out short-term fluctuations and variability in the dataset. The smoothed curves are visualized below and can be compared to the raw time series above.

In [57]:
smoothed_pygmy_owl = pygmy_owl.copy()
smoothed_pygmy_owl['n_sightings'] = pygmy_owl['n_sightings'].rolling(window=30, center=True).mean()
plot_time_series(data=smoothed_pygmy_owl, title='Pygmy owl (Sperlingskauz) sightings in grid 50kmE4100N2700 (smoothed)')

In [8]:
smoothed_warbler = warbler.copy()
smoothed_warbler['n_sightings'] = warbler['n_sightings'].rolling(window=30, center=True).mean()
plot_time_series(data=smoothed_warbler, title='Icterine warbler (Gelbspötter) sightings in grid 50kmE4350N3250 (smoothed)')

# 
<p style="background-color:#4A3228;color:white;font-size:240%;border-radius:10px 10px 10px 10px;"> &nbsp; 2️⃣ Change point detection </p>

Change point detection is a statistical method used to identify shifts in the probability distribution of a stochastic process or time series. This can help in identifying concept drift by signaling when the data distribution changes significantly, allowing for informed decision-making and model updates. 

The following four methods for change point detection will be applied in this notebook. Unlike the reports by EvidentlyAI in the previous notebooks, these change point detection methods continuously monitor the data for changes, meaning they detect changes continuously rather than comparing data distributions on a yearly or periodic basis.

**`1️⃣ ADWIN (ADaptive WINdowing):`** This algorithm maintains a variable-size sliding window of data points and continuously monitors statistical properties. When a change is detected, the window size is adjusted to adapt to the new data distribution. The key idea is to maintain a window of variable size, where the window grows when no change is detected and shrinks when a change is detected. This allows to handle both gradual and sudden changes. 

**`2️⃣ KSWIN (Kolmogorov-Smirnov WINdowing):`** Unlike ADWIN, KSWIN uses kernel density estimation within a sliding window to detect changes in the data's underlying distribution.

**`3️⃣ Page Hinkley Test:`** This sequential analysis technique efficiently detects changes in the average of a Gaussian process by considering the magnitude of the change.

**`4️⃣ 2nd Derivative:`** This straightforward method involves computing the second derivative of a time series to detect abrupt changes in the trend, which can indicate a change point. Despite its simplicity, it is applied for detecting change points in various domains, such as `sources here`.

### The pygmy owl: Detecting habitat drifts
Below is the time series of sightings of the pygmy owl, with sighting frequency on the y-axis. Upon examining the curves, it is noticeable that the pygmy owl was never sighted in the specified grid around Basel (CH) until 2019. Therefore, according to our understanding, a concept drift (or anomaly) would occur in early 2019, as the sighting of a pygmy owl in this grid was previously very unlikely; with multiple sightings in the spring of 2019, reviewers would consider the submission of a sighting as more likely and may approve it, hence there is a concept drift as the label of the input variable *Sighting of a pygmy owl* shifts from *implausible* to *rather plausible*. Since the sightings subsequently decrease, this increase could arguably also be considered an anomaly; however, this notebook is initially focused on identifying change points. A distinction will be made in subsequent notebooks. Another concept drift could be that the sightings generally gradually increase over the years and in the winter of 2021, where the sightings do not decrease to zero as in previous years.

As we can see, all algorithms recognize some change points, including the change that in the fall of 2021, the sightings do not decrease as in previous years. `-> Algorithm comparison here` Many change points are, according to their definition, set at sudden increases and decreases. The accuracy and timeliness of the set change points appear reasonably accurate. The change that the sightings do not decrease in the fall of 2021 and the bird was observed throughout the winter is also marked.

However, we see that many increases and decreases can be attributed to the seasonality of bird sightings. As Simon showed in his EDA notebook, user activity on the portals ornitho.de and ornitho.ch decreases significantly in the colder months. Therefore, sightings of resident birds such as the pygmy owl also decrease, even though the bird still resides in Germany and Switzerland. Hence, an increase in the spring and decrease in the fall is a normal seasonally based characteristic of our time series, and not an indication of an anomaly or drift (with the exception if these change points shift over the years; for example, an increased sighting of the bird in the winter of 2021). Since a change point is simply defined as an increase or decrease in the data, our algorithms cannot differentiate between seasonally induced and "real" drifts and anomalies by definition. Further considerations and possible future directions on this matter can be found in Chapter 3️⃣.

The situation becomes even clearer when considering the icterine warbler, a migratory bird with high seasonality (see next chapter).

In [67]:
change_detectors = {'ADWIN': drift.ADWIN(clock=1),
                    'KSWIN': drift.KSWIN(window_size=100, stat_size=7), 
                    'Page Hinkley': drift.PageHinkley(threshold=0.1, min_instances=3, delta=0.0005)}

In [68]:
stream = smoothed_pygmy_owl['n_sightings'].values

change_points = {}
for algo_name, change_detector in change_detectors.items():
    cps = []
    for i, val in enumerate(stream):
        change_detector.update(val)
        if change_detector.drift_detected:
            cps.append(i)
    change_points[algo_name] = cps
    print(algo_name, cps)

plot_change_points(data=smoothed_pygmy_owl, 
                   change_points_dict=change_points,
                   title='Pygmy owl (Sperlingskauz) sightings in grid 50kmE4100N2700')

ADWIN []
KSWIN [393, 493, 669, 793, 982, 1116, 1216, 1332, 1432, 1534, 1634, 1741]
Page Hinkley []


Im Fall des Mann-Kendall Test mit saisonaler Komponente berücksichtigt man den saisonalen Charakter der Reihe. Anders ausgedrückt für monatliche Daten mit einer Saisonalität von 12 Monaten, wird nicht versucht zu überprüfen ob ein globales Ansteigen der Reihe vorliegt, sondern einfach ob von einem Januar zum nächsten, von einem Februar zum nächsten, und so weiter eine Tendenz festgestellt werden kann.

In [51]:
import pymannkendall as mk
print(mk.seasonal_test(smoothed_pygmy_owl['n_sightings'], period=12))  # monthly seasonality

Seasonal_Mann_Kendall_Test(trend='increasing', h=True, p=0.0, z=23.967210494334278, Tau=0.37044896876450695, s=11172.0, var_s=217244.66666666663, slope=0.00505050505050505, intercept=-0.05534188034188033)


### The icterine warbler: Detecting date drifts
With the icterine warbler, we can see the clear influence of the seasonality of the data on the algorithms. All change points are, by their definition, correctly and with little latency set, which underscores the ability of the algorithms to detect change points in our data.

However, according to our definition, these change points are not a reason for an anomaly or drift, as they simply reflect the normal migration behavior of the bird.

In [60]:
change_detectors = {'ADWIN': drift.ADWIN(),
                    'KSWIN': drift.KSWIN(), 
                    'Page Hinkley': drift.PageHinkley()}

In [34]:
from skmultiflow.drift_detection.adwin import ADWIN as ADWIN_skmultiflow
adwin = ADWIN_skmultiflow(delta=1)

for i, val in enumerate(stream):
    adwin.add_element(val)
    if adwin.detected_change():
        print('Change detected in data at index:', i)

In [61]:
stream = smoothed_warbler['n_sightings'].values

change_points = {}
for algo_name, change_detector in change_detectors.items():
    cps = []
    for i, val in enumerate(stream):
        change_detector.update(val)
        if change_detector.drift_detected:
            cps.append(i)
    change_points[algo_name] = cps

plot_change_points(data=smoothed_warbler, 
                   change_points_dict=change_points, 
                   title='Icterine warbler (Gelbspötter) sightings in grid 50kmE4350N3250')

In the following figure, the years have been overlaid instead of being represented as a continuous time series. For the sake of clarity, this figure only displays the change points of the KSWIN algorithm.

Here, it is evident how the arrival and departure times of the bird shift over the years. As already shown in `03_date_drift.ipynb`, these shifts are not gradual, but likely randomly fluctuating due to weather conditions. However, in this case, it may not be the occurrence of a change point, but rather *the shifting of recurring change points over the years* that could be defined as drift. Further considerations on this matter can be found in the discussion in Chapter 3️⃣.

In [28]:
plot_change_points_per_year(data=smoothed_warbler, 
                   change_points=change_points_adwin_river, 
                   title='Icterine warbler (Gelbspötter) sightings in grid 50kmE4350N3250', 
                   legend_title='detected change point (KSWIN)',
                   show_first_sighting=False)

# 
<p style="background-color:#4A3228;color:white;font-size:240%;border-radius:10px 10px 10px 10px;"> &nbsp; 3️⃣ Discussion </p>

## Strenghts of the applied algorithms
The algorithms can effectively detect abrupt as well as gradual changes in the time series with satisfactory accuracy and low latency.

## Limitations of continuous change point detection
As revealed in this notebook, continuous monitoring using sliding windows when applied to bird sighting data presents a challenge. While all algorithms can effectively identify increases or decreases in sightings to a greater or lesser extent, the identified concept drifts can mostly be attributed to the seasonality of the sightings, influenced by the migration of a bird or the decrease in user activity in the winter. This seasonality is a normal behavior of bird sightings and does not represent a shift in bird behavior regarding migration, habitat, or population density. However, the algorithms lack this knowledge and the ability to distinguish drifts in bird populations from seasonally induced drifts.

## Possible directions
For the seasonal characteristic of the sighting data, ways need to be found to address the yearly periodic increase and decrease in sightings and to differentiate "real" concept drifts from seasonal events. This requires a need for definitions of what we understand as a real concept drift. In addition, ways must be devised to algorithmically separate these drifts.

One possibility could be to refrain from continuous monitoring and instead return to comparing the sighting patterns per species and grid over entire yearly periods, similar to the approach in notebooks `02_habitat_drift.ipynb` and `03_date_drift.ipynb`. The advantage here is that seasonality is present in both distributions and thus, if the seasonality remains the same, it is not considered as drift. If a drift in the distribution is detected, the change points could be more accurately determined through statistical analysis to identify the day the drift occurred. With this, in the example of the icterine warbler, the shifting of recurring change points, representing a shift in arrival and departure, could be considered as a gradual drift over the years, rather than considering the bird's arrival as a drift compared to its absence in the winter. Real-time analyses could still be possible. For example, in June of a year, the data from the previous year up to June could be considered, and a drift analysis could be performed within this 6-month period.